# <center>STUDENT AT PARYS SACLAY 25 OCTOBER 2020</center>#
# <center> Programming Project Document Similarity</center>#
#  NAMES: PANONGBENE JEAN MOHAMED SAWADOGO #
# Email: amet1900@gmail.com #
# Github: https://github.com/Panongbene #

In [1]:
import nltk
import numpy as np
import pandas as pd
from requests import get
from random import shuffle
from bs4 import BeautifulSoup
from copy import deepcopy

In [47]:
# use this variable for the tokenize the text
tokenizer = nltk.RegexpTokenizer(r'\w+')

# name File Text
name_file_text = "tweets.txt"

# the threshold value
threshold_simmilarity = 0.05

# the number of permutation function
number_hash_function = 19

# the size of shingle
size_shingle = 8

In [48]:
# the list containing each line of the doc
array_data = list()

# open the file and build the list
file1 = open(name_file_text,"r")
for i in file1.readlines():
    array_data.append(i.split("\r\n")[0])
file1.close() 

# creation of a dataframe containing the text
data_frame_text_1 = pd.DataFrame(data=array_data, columns=['Text_line'])

In [49]:
# text data header
data_frame_text_1.head()

,Text_line
0,@stellargirl I loooooooovvvvvveee my Kindle2. ...
1,Reading my kindle2... Love it... Lee childs i...
2,"Ok, first assesment of the #kindle2 ...it fuck..."
3,@kenburbary You'll love your Kindle2. I've had...
4,@mikefish Fair enough. But i have the Kindle2...


In [86]:
#----------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#
def jaccard_similarity(text_1, text_2):
    """This function take two list of word and return the jaccard similarity"""
    text_1_inter_text_2 = 0
    
    for i in text_1:
        if i in text_2:
            text_1_inter_text_2+=1
            
    return float(text_1_inter_text_2)/float(len(text_1)+len(text_2)-text_1_inter_text_2)
#----------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#

def jaccard_similarity_(kkkk, s1,s2):
    """Another version of jaccard similarity"""
    valeur = 0
    for kj in range(len(kkkk)):
        if(kkkk[kj][s1] == kkkk[kj][s2]):
            valeur+=1
    
    return valeur
#----------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#
def preprocessing_doc(array_data):
    """We use this function for build a dictionary where the keys is the ids of document and the values is the 
    texts"""
    
    # define a dictionnary that the function will return 
    dictionnary_doc_1 = dict()
    
    #tokekenize the doc
    for i in range(len(array_data)):
        array_text_tokenize = tokenizer.tokenize(array_data[i].lower())
        text = ""
        for k in array_text_tokenize:
            text+=k
        
        dictionnary_doc_1[i]=text
        
    return dictionnary_doc_1
#----------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#
def k_shingling_caractere(dictionnary_doc, size_k):
    """This function take a dictionary where the keys are the ids of docs and the values 
    are the texts containing the essential words of the document. The second parameter is
    the length of shingling"""
    
    shingling = dict()
    
    key = 0
    for i in dictionnary_doc:
        kkkk = list()
        text = dictionnary_doc[i]
        
        if(len(text)>=size_k):
            j = 0;
            
            while(j+size_k < len(text)):
                kkk = ""
                for k in range(j, j+size_k):
                    kkk+=text[k]
                j+=1
                kkkk.append(kkk)
            
            kkk = ""
            for k in range(j, len(text)):
                kkk+=text[k]
            kkkk.append(kkk)
            
            shingling[key]=kkkk
            key+=1
        else:
            kkk = ""
            for k in range(len(text)):
                kkk+=text[k]
            kkkk.append(kkk)
            shingling[key]=kkkk
            key+=1
            
    
    # the dictionnary will contain all different k-shingle
    shingle_hash = list()
    
    # building the shingle_hash
    k = 0
    for i in shingling:
        for j in shingling[i]:
            if j not in shingle_hash:
                shingle_hash.append(j)
            
    return shingling, shingle_hash
#----------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#
def build_min_hashing_matrix(dictionnary_doc, shingle_hash, permutation_functions):
    """this function is used to build the signature matrix"""
    
    signature_matrix = dict()
 
    for k in range(len(permutation_functions)):
        value_row = list()

        #--------------------------------------------
        for i in dictionnary_doc:
            for j in permutation_functions[k]:
                if(j in dictionnary_doc[i]):
                    value_row.append(j)
                    break
        #--------------------------------------------            
        
        signature_matrix[k] = value_row
        
    return signature_matrix
#----------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#
def build_similarity_matrix(signature_matrix):
    """This function take minhashing matrix and return the signature matrix"""
    
    similarity_matrix = dict()
    
    
    for i in range(len(signature_matrix[0])-1):
        
        array_tampon = list()
        
        for j in range(i,len(signature_matrix[0])):
            array_tampon.append(jaccard_similarity_(signature_matrix, i,j)/number_hash_function)
        
        similarity_matrix[i]=array_tampon
            
    
    return similarity_matrix
#----------------------------------------------------------------------------------------------------------------#
#----------------------------------------------------------------------------------------------------------------#

In [6]:
#-------------------------------------------------------------------------------------------------------------#
#-------------------------------------------------------------------------------------------------------------#

In [51]:
#dictionnary_doc = preprocessing_document(array_data)
dictionnary_doc_1 = preprocessing_doc(array_data)

In [52]:
# creation of k-shingling
k_shingling_carac, shingle_hash = k_shingling_caractere(dictionnary_doc_1, size_shingle)

In [53]:
# we create number_perm of permutation function
permutation_functions = dict()
for i in range(number_hash_function):
    permutation_functions[i] = deepcopy(shingle_hash)
    shuffle(permutation_functions[i]) 

In [59]:
# build the mini hashing matrix
signature_matrix = build_min_hashing_matrix(k_shingling_carac, shingle_hash, permutation_functions)

In [ ]:
#Build the similarity matrix
testest = build_similarity_matrix(signature_matrix)

In [11]:
#-------------------------------------------------------------------------------------------------------------#
#-------------------------------------------------------------------------------------------------------------#

In [78]:
for i in range(len(signature_matrix[0])-1):
    for j in range(i+1,len(signature_matrix[0])):
        if jaccard_similarity_(signature_matrix, i,j)>0:
            print(i,"  ==  ",j,"  ==  ",jaccard_similarity_(signature_matrix, i,j)/number_hash_function)
            print(jaccard_similarity(k_shingling_carac[i], k_shingling_carac[j]))

0   ==   455   ==   0.10526315789473684
0.0390625
1   ==   455   ==   0.05263157894736842
0.023809523809523808
1   ==   464   ==   0.05263157894736842
0.03571428571428571
1   ==   466   ==   0.05263157894736842
0.012048192771084338
2   ==   3   ==   0.05263157894736842
0.007575757575757576
2   ==   265   ==   0.10526315789473684
0.03418803418803419
3   ==   265   ==   0.05263157894736842
0.016574585635359115
4   ==   265   ==   0.05263157894736842
0.06349206349206349
8   ==   234   ==   0.05263157894736842
0.018691588785046728
10   ==   12   ==   0.10526315789473684
0.1782178217821782
10   ==   137   ==   0.05263157894736842
0.023952095808383235
10   ==   246   ==   0.05263157894736842
0.06363636363636363
10   ==   261   ==   0.05263157894736842
0.025974025974025976
10   ==   411   ==   0.10526315789473684
0.11764705882352941
12   ==   16   ==   0.05263157894736842
0.014184397163120567
12   ==   133   ==   0.05263157894736842
0.014285714285714285
12   ==   246   ==   0.0526315789473684

101   ==   102   ==   0.05263157894736842
0.03968253968253968
101   ==   121   ==   0.05263157894736842
0.007142857142857143
101   ==   242   ==   0.05263157894736842
0.01020408163265306
101   ==   270   ==   0.05263157894736842
0.00980392156862745
102   ==   121   ==   0.05263157894736842
0.006535947712418301
102   ==   242   ==   0.05263157894736842
0.009009009009009009
102   ==   270   ==   0.05263157894736842
0.008695652173913044
105   ==   293   ==   0.05263157894736842
0.007246376811594203
106   ==   269   ==   0.05263157894736842
0.024793388429752067
106   ==   348   ==   0.21052631578947367
0.06481481481481481
106   ==   444   ==   0.10526315789473684
0.022222222222222223
107   ==   348   ==   0.2631578947368421
0.4444444444444444
108   ==   109   ==   0.05263157894736842
0.12941176470588237
108   ==   110   ==   0.15789473684210525
0.10679611650485436
108   ==   111   ==   0.05263157894736842
0.20634920634920634
108   ==   350   ==   0.05263157894736842
0.07692307692307693
108

132   ==   361   ==   0.05263157894736842
0.10112359550561797
132   ==   362   ==   0.15789473684210525
0.15254237288135594
132   ==   363   ==   0.15789473684210525
0.15517241379310345
132   ==   364   ==   0.05263157894736842
0.06976744186046512
132   ==   365   ==   0.10526315789473684
0.14492753623188406
132   ==   366   ==   0.10526315789473684
0.10588235294117647
132   ==   367   ==   0.05263157894736842
0.09433962264150944
132   ==   368   ==   0.10526315789473684
0.13432835820895522
132   ==   369   ==   0.05263157894736842
0.09574468085106383
132   ==   370   ==   0.10526315789473684
0.09782608695652174
132   ==   371   ==   0.10526315789473684
0.11688311688311688
132   ==   372   ==   0.10526315789473684
0.1111111111111111
133   ==   134   ==   0.05263157894736842
0.0736196319018405
133   ==   135   ==   0.10526315789473684
0.07317073170731707
133   ==   136   ==   0.05263157894736842
0.06206896551724138
133   ==   246   ==   0.05263157894736842
0.014492753623188406
133   == 

152   ==   153   ==   0.05263157894736842
0.06299212598425197
152   ==   154   ==   0.10526315789473684
0.07476635514018691
152   ==   211   ==   0.10526315789473684
0.05925925925925926
152   ==   212   ==   0.05263157894736842
0.023809523809523808
152   ==   214   ==   0.05263157894736842
0.02608695652173913
152   ==   385   ==   0.05263157894736842
0.01694915254237288
152   ==   387   ==   0.05263157894736842
0.026785714285714284
152   ==   388   ==   0.10526315789473684
0.029850746268656716
152   ==   389   ==   0.05263157894736842
0.027777777777777776
152   ==   390   ==   0.10526315789473684
0.022900763358778626
152   ==   391   ==   0.15789473684210525
0.09302325581395349
152   ==   392   ==   0.10526315789473684
0.02586206896551724
152   ==   393   ==   0.05263157894736842
0.028846153846153848
152   ==   394   ==   0.05263157894736842
0.018404907975460124
152   ==   396   ==   0.05263157894736842
0.02112676056338028
152   ==   398   ==   0.05263157894736842
0.03333333333333333
1

176   ==   496   ==   0.05263157894736842
0.006578947368421052
177   ==   292   ==   0.05263157894736842
0.021739130434782608
179   ==   180   ==   0.05263157894736842
0.02564102564102564
182   ==   420   ==   0.05263157894736842
0.017543859649122806
183   ==   201   ==   0.05263157894736842
0.017543859649122806
183   ==   233   ==   0.05263157894736842
0.023255813953488372
183   ==   240   ==   0.05263157894736842
0.030303030303030304
183   ==   294   ==   0.05263157894736842
0.02247191011235955
183   ==   311   ==   0.05263157894736842
0.027777777777777776
183   ==   328   ==   0.05263157894736842
0.024096385542168676
183   ==   345   ==   0.05263157894736842
0.024691358024691357
183   ==   394   ==   0.05263157894736842
0.014598540145985401
183   ==   414   ==   0.05263157894736842
0.019417475728155338
183   ==   423   ==   0.42105263157894735
0.3389830508474576
183   ==   446   ==   0.05263157894736842
0.019801980198019802
183   ==   467   ==   0.05263157894736842
0.014705882352941

232   ==   246   ==   0.05263157894736842
0.03409090909090909
232   ==   411   ==   0.05263157894736842
0.047619047619047616
233   ==   240   ==   0.05263157894736842
0.030303030303030304
233   ==   246   ==   0.10526315789473684
0.023255813953488372
233   ==   261   ==   0.10526315789473684
0.015748031496062992
233   ==   262   ==   0.05263157894736842
0.019801980198019802
233   ==   266   ==   0.05263157894736842
0.017094017094017096
233   ==   269   ==   0.10526315789473684
0.021505376344086023
233   ==   279   ==   0.05263157894736842
0.014084507042253521
233   ==   294   ==   0.15789473684210525
0.03409090909090909
233   ==   300   ==   0.05263157894736842
0.0273972602739726
233   ==   311   ==   0.10526315789473684
0.027777777777777776
233   ==   325   ==   0.05263157894736842
0.02040816326530612
233   ==   328   ==   0.15789473684210525
0.036585365853658534
233   ==   345   ==   0.05263157894736842
0.0375
233   ==   351   ==   0.10526315789473684
0.022988505747126436
233   ==   

274   ==   415   ==   0.21052631578947367
0.051470588235294115
274   ==   416   ==   0.15789473684210525
0.03763440860215054
274   ==   424   ==   0.10526315789473684
0.04666666666666667
274   ==   443   ==   0.15789473684210525
0.038461538461538464
274   ==   448   ==   0.15789473684210525
0.04294478527607362
274   ==   492   ==   0.15789473684210525
0.04487179487179487
279   ==   294   ==   0.05263157894736842
0.013793103448275862
279   ==   300   ==   0.05263157894736842
0.015503875968992248
279   ==   325   ==   0.05263157894736842
0.012987012987012988
279   ==   328   ==   0.05263157894736842
0.021739130434782608
279   ==   351   ==   0.05263157894736842
0.02112676056338028
279   ==   386   ==   0.05263157894736842
0.010471204188481676
279   ==   394   ==   0.05263157894736842
0.010362694300518135
279   ==   411   ==   0.05263157894736842
0.014388489208633094
279   ==   414   ==   0.05263157894736842
0.012578616352201259
279   ==   446   ==   0.05263157894736842
0.0127388535031847

347   ==   470   ==   0.10526315789473684
0.032520325203252036
350   ==   352   ==   0.05263157894736842
0.07042253521126761
351   ==   352   ==   0.05263157894736842
0.11363636363636363
351   ==   386   ==   0.05263157894736842
0.014705882352941176
351   ==   394   ==   0.05263157894736842
0.014492753623188406
351   ==   411   ==   0.05263157894736842
0.023809523809523808
351   ==   414   ==   0.05263157894736842
0.019230769230769232
351   ==   446   ==   0.05263157894736842
0.0196078431372549
351   ==   447   ==   0.05263157894736842
0.016129032258064516
351   ==   482   ==   0.10526315789473684
0.02531645569620253
355   ==   367   ==   0.05263157894736842
0.018018018018018018
356   ==   357   ==   0.05263157894736842
0.007936507936507936
356   ==   358   ==   0.05263157894736842
0.008130081300813009
357   ==   358   ==   0.10526315789473684
0.009615384615384616
361   ==   362   ==   0.10526315789473684
0.1232876712328767
361   ==   363   ==   0.10526315789473684
0.125
361   ==   364

394   ==   467   ==   0.05263157894736842
0.0106951871657754
394   ==   482   ==   0.05263157894736842
0.015503875968992248
395   ==   396   ==   0.10526315789473684
0.05732484076433121
395   ==   398   ==   0.15789473684210525
0.07547169811320754
396   ==   398   ==   0.2631578947368421
0.11627906976744186
396   ==   452   ==   0.05263157894736842
0.03614457831325301
398   ==   452   ==   0.05263157894736842
0.02564102564102564
399   ==   414   ==   0.05263157894736842
0.02857142857142857
399   ==   423   ==   0.05263157894736842
0.025
399   ==   429   ==   0.05263157894736842
0.019801980198019802
399   ==   446   ==   0.05263157894736842
0.019230769230769232
399   ==   465   ==   0.05263157894736842
0.013986013986013986
401   ==   415   ==   0.10526315789473684
0.1
401   ==   416   ==   0.10526315789473684
0.05343511450381679
401   ==   424   ==   0.10526315789473684
0.07368421052631578
401   ==   443   ==   0.10526315789473684
0.05511811023622047
401   ==   448   ==   0.105263157894

In [87]:
testest = build_similarity_matrix(signature_matrix)

In [88]:
for i in range(len(testest)):
    if(testest[0][i]>0):
        print(i)

0
455


In [ ]:
def locality_sensitive_hashing(signature_matrix,number_of_bande,number_of_buckets):
    """This function """
    
    
    